### Box Interactive Config

* data coming from the box is used by the **ISI_Dashboard** for realtime visualization of variables that are important to the experimenter. 
* here we construct messages that are sent interactively to the box over the instrument loop.
* we can manipulate these messages inorder to test and change the function of the **ISI_Dashboard**

### MockBox usage description: 

1. a client connects to the MockBox server
2. the MockBox Server sends a message to the client asking for it's "ID_REQ"
    1. this happens once and only once
3. if the "ID_REQ" == 1, then the client subscribes to JSON messages indefinitely.

### run the server

* to initiate and run the server, run the designated cell

#### stimulation system architecture

* stim_req_packets are sent in, at whatever frequency (could be 1khz)
* each time a packet is recieved, it overwrites a dictionary in memory which requests 


#### there is not 1 message, but a set of messages types that a logger subscribes to. These are: 

stim_event_json 
stim_ack_json
stim_req_json
soh_json

ID_REQ

#### all of these messages have the following structure.

{
"client_ip": str
"client_id": int OR "N/A"
"msg_type": str
"msg": str - sometimes JSON structured
}

this facilitates handling of different behaviors for different message types by first reading the 'msg_type' parameter.


#### understanding how electrode arrays are indexed:

arrays are indexed by a single number, either 1-32 or 129 - 161
1-32 is the first set of electrodes,  129-161 is the second
electrode arrays are numbered from top to bottom, and from left to right
so bottom left corner is 1,one to the right is 9, etc. 
you can figure out if this is something you want ot modify, or just stick with it.


#### how the stim packets are organized

within the stim_event_json packet's msg we have:

{
elecCath: int OR list of ints,
elecAno: int OR list of ints,
amp: int,
freq: int,
pulseWidth: int,
isContinuous: int
} 

the state of the ground and reference nodes are trickled back through the SOH.


#### 

* there is no state stored on the box
* multiple stim packets can be appended to the stim_req_json, these multiple packets constitute the full state configuration of the box, minus the ground configuration packets.
* ground and reference can be viewed in the state of health, but the 
* stim_req_packet contains the full anode.



#### more notes:

* the box won't trickle back the json to the logger that I need unless stim sequences are actually sent. 
* everything that I will need from a graphing perspective will come through my connection to the logger (except for DJ stuff, which is it's own webserver)

 

### configure messages that will be sent to the box

In [16]:
# configure ground electrodes
refs = {}
refs['ground']   = [30]
refs['reference'] = [31]  

# configure stim packet
sd1 = {}
sd1["elecCath"] = [1,12]
sd1["elecAno"] = [3,4]
sd1["amp"] = 300
sd1["freq"] = 20 
sd1["pulseWidth"] = 100 
sd1["isContinuous"] = 0

sd2 = {}
sd2["elecCath"] = [7,8]
sd2["elecAno"] = [9,10]
sd2["amp"] = 200
sd2["freq"] = 30
sd2["pulseWidth"] = 150 
sd2["isContinuous"] = 0

#stim clear dictionary
sdc = {}
sdc["elecCath"] = []
sdc["elecAno"] = []
sdc["amp"] = 0
sdc["freq"] = 20 
sdc["pulseWidth"] = 100 
sdc["isContinuous"] = 0

stimPacket = [sd1, sd2]
Clear = False 
if Clear:
    stimPacket = [sdc]





### Send packets to Box

In [17]:
import websockets
import asyncio
import json

async def SendStim():
    ip_addr = "192.168.42.1"
    port = "7890"
    url = "ws://" + ip_addr + ":" + port

    async with websockets.connect(url) as ws:
        # connect and provide ID - determine the class of listener.
        while True:                                     # ID check loop
            msg = await ws.recv()                       # Non-blocking, waits for a new message to arrive from the server
            if msg == "ID_REQ":                         # If the message is an ID Request
                my_client_id = 11                        # Fetch this client's ID, in this case it's hardcoded
                await ws.send(str(my_client_id))        # Non-blocking, sends the client ID
                print("connected Successfully as stim")
                break                                   # out of ID check loop
        
        #send a stim requests forever 
        while True: 
            await ws.send(json.dumps(refs))
            await ws.send(json.dumps(stimPacket))
            await asyncio.sleep(1)


loop = asyncio.get_event_loop()
serverTask = loop.create_task(SendStim())
serverTask

<Task pending name='Task-24' coro=<SendStim() running at C:\Users\adawsone\AppData\Local\Temp\ipykernel_1084\2108555202.py:5>>

connected Successfully as stim


### Cancel the server

* run the cell below to cancel the server task

In [15]:
serverTask.cancel()

False

In [1]:
import websockets
import asyncio
import json

class Storage:
    def __init__(self):
        self.msg = ""

storage = Storage()

async def listen():
    ip_addr = "192.168.42.1"
    port = "7890"
    url = "ws://" + ip_addr + ":" + port

    async with websockets.connect(url) as ws:
        # connect and provide ID - determine the class of listener.
        while True:                                     # ID check loop
            msg = await ws.recv()                       # Non-blocking, waits for a new message to arrive from the server
            if msg == "ID_REQ":                         # If the message is an ID Request
                my_client_id = 1                        # Fetch this client's ID, in this case it's hardcoded
                await ws.send(str(my_client_id))        # Non-blocking, sends the client ID
                print("connected Successfully as logger")
                break                                   # out of ID check loop
        
        #read a single message then break
        while True:
            msg = await ws.recv()                       # Non-blocking, waits for a new message to arrive from the server
            storage.msg = msg
            break
        #await ws.close()


#entrypoint
#await main()
loop = asyncio.get_event_loop()
serverTask = loop.create_task(listen())
serverTask

<Task pending name='Task-4' coro=<listen() running at C:\Users\adawsone\AppData\Local\Temp\ipykernel_14924\3521875238.py:11>>

connected Successfully as logger


In [ ]:
serverTask.cancel()

In [1]:
14 % 8

6

In [3]:
import numpy as np
ppm = np.array([[ 8 , 16 , 24 , 32 ],
                [ 7 , 15 , 23 , 31 ],
                [ 6 , 14 , 22 , 30 ],
                [ 5 , 13 , 21 , 29 ],
                [ 4 , 12 , 20 , 28 ],
                [ 3 , 11 , 19 , 27 ],
                [ 2 , 10 , 18 , 26 ],
                [ 1 ,  9 , 17 , 25 ]])

In [15]:
pmap = map(lambda a :  7 - ((a-1) % 8),ppm)
list(pmap)

[array([0, 0, 0, 0]),
 array([1, 1, 1, 1]),
 array([2, 2, 2, 2]),
 array([3, 3, 3, 3]),
 array([4, 4, 4, 4]),
 array([5, 5, 5, 5]),
 array([6, 6, 6, 6]),
 array([7, 7, 7, 7])]

In [13]:
pmap = map(lambda a : np.floor((a-1) / 8),ppm)
list(pmap)

[array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.])]